In [15]:
dataset = "dataset_1"

import numpy as np

tp = np.load("data/datasets/{}/topology.npy".format(dataset))

device_num = len(tp)

device_dict = dict()

In [16]:
for index, row in enumerate(tp):
    device_dict[index] = [i for i in range(device_num) if row[i] == 1]

In [17]:
import numpy as np
cp = np.load("data/datasets/{}/causal_prior.npy".format(dataset))

alarm_num = len(cp)

alarm_dict = dict()

In [18]:
for index, row in enumerate(cp):
    alarm_dict[index] = [i for i in range(alarm_num) if row[i] != 0]

In [19]:
import pandas as pd
import numpy as np

alarms = pd.read_csv('data/datasets/{}/alarm.csv'.format(dataset))
cp = np.load('data/datasets/{}/causal_prior.npy'.format(dataset))

alarms.sort_values(by="start_timestamp" , inplace=True, ascending=True) 

In [20]:
data = alarms.to_dict(orient="records")


In [21]:
record_stack = list()


while data:
    record = data.pop(0)
    start_timestamp = record["start_timestamp"]
    alarm_id = record["alarm_id"]
    device_id = record["device_id"]
    index = 0
    while True:
        if index == len(data):
            break
        if data[index]["start_timestamp"] - start_timestamp > 1:
            break
        if data[index]["device_id"] in device_dict[device_id] and data[index]["alarm_id"] in alarm_dict[alarm_id]:
            if alarm_id != data[index]["alarm_id"] and cp[alarm_id][data[index]["alarm_id"]] == -1 and cp[data[index]["alarm_id"]][alarm_id] != 1:
                cp[alarm_id][data[index]["alarm_id"]] = 1
        index += 1

cp[cp == -1] = 0
np.save('./submission/bfs_1s/{}_graph_matrix.npy'.format(dataset),cp)
        
        